In [4]:
import numpy as np
import glob
import re

def read_psq (path):
  pattern = re.compile('[0-9]+,[0-9]+,[0-9]+')
  ret = []
  with open(path, 'rt') as f:
    for line in f.readlines():
      if pattern.match(line):
        ret.append(list(map(lambda x: int(x) - 1, line.split(',')))[0: 2])
  
  return ret


In [5]:
def kibo_to_input_list (kibo):
  ret = []
  cur_state = np.zeros((20, 20), dtype=np.int8)
  for i, v in enumerate(kibo):
    ret.append(np.array(cur_state))
    cur_state[v[1]][v[0]] = (1 if i % 2 == 0 else 2)
  return ret

def kibo_to_action_list (kibo):
  ret = []
  for v in kibo:
    ret.append(np.array([v[0], v[1]], np.int32))
  return ret

def kibo_to_reward_list (kibo):
  ret = []
  for i, _ in kibo:
    if len(kibo) >= 400:
      ret.append(0)
    else:
      ret.append(1 if (i % 2) != (len(kibo) % 2) else -1)
  return ret

def kibo_to_turn_list (kibo):
  ret = []
  for i, _ in kibo:
    ret.append(i % 2)
  return ret


In [6]:
from itertools import permutations

def rotate90_gomoku_pos (pos):
  return np.array([pos[1], 19 - pos[0]], dtype=np.int32)

def mirror_y_gomoku_pos (pos):
  return np.array([pos[0], 19 - pos[1]], dtype=np.int32)

def get_mirror (num):
  return 19 - num


In [141]:
import re
import os
from tqdm import tqdm

pattern = re.compile('freestyle[0-9]+$', re.RegexFlag.IGNORECASE)

psqs = []
for i in [2017, 2018, 2019, 2020, 2021]:
  for item in os.listdir('./dataset/gomocup{}results'.format(i)):
    if pattern.match(item):
      psqs += glob.glob('./dataset/gomocup{}results/{}/*.psq'.format(i, item))


In [7]:
import os

def save_processed_datas (*, input_datas=[], action_datas=[], reward_datas=[], turn_datas=[], file_num=0):
  os.mkdir('./dataset/processed_datas/no{}'.format(file_num))
  np.save('./dataset/processed_datas/no{}/inputs.npy'.format(file_num), np.array(input_datas, dtype=np.int8))
  np.save('./dataset/processed_datas/no{}/actions.npy'.format(file_num), np.array([v[0] + v[1] * 20 for v in action_datas], dtype=np.int32))
  np.save('./dataset/processed_datas/no{}/rewards.npy'.format(file_num), np.array(reward_datas, dtype=np.int8))
  np.save('./dataset/processed_datas/no{}/turns.npy'.format(file_num), np.array(turn_datas, dtype=np.int8))


In [8]:
def transform_board (*, input_list, action_list, operation):
  return operation(input_list, action_list)

def rotate90 (input_list, action_list):
  input_list_ret = []
  action_list_ret = []
  for input, action in zip(input_list, action_list):
    input_list_ret.append(np.rot90(input))
    action_list_ret.append(rotate90_gomoku_pos(action))
  return input_list_ret, action_list_ret

def mirror (input_list, action_list):
  input_list_ret = []
  action_list_ret = []
  for input, action in zip(input_list, action_list):
    input_list_ret.append(np.flip(input, axis=0))
    action_list_ret.append(mirror_y_gomoku_pos(action))
  return input_list_ret, action_list_ret

def identity_operation (input_list, action_list):
  return input_list, action_list

operation_list = [rotate90 for _ in range(3)] + [mirror] + [rotate90 for _ in range(3)] + [identity_operation]


In [146]:
from tqdm import tqdm

BLOCK_SIZE_PER_FILE = 1000000

input_datas = []
action_datas = []
reward_datas = []
turn_datas = []

file_idx = 0

for psq in tqdm(psqs):
  kibo = read_psq(psq)
  input_list = kibo_to_input_list(kibo)
  action_list = kibo_to_action_list(kibo)
  reward_list = kibo_to_reward_list(kibo)
  turn_list = kibo_to_turn_list(kibo)
  for operation in operation_list:
    input_datas += input_list
    action_datas += action_list
    reward_datas += reward_list
    turn_datas += turn_list
    input_list, action_list = transform_board(input_list=input_list, action_list=action_list, operation=operation)

  while len(input_datas) >= BLOCK_SIZE_PER_FILE:
    save_processed_datas(
      file_num=file_idx,
      input_datas=input_datas[0: BLOCK_SIZE_PER_FILE],
      action_datas=action_datas[0: BLOCK_SIZE_PER_FILE],
      reward_datas=reward_datas[0: BLOCK_SIZE_PER_FILE],
      turn_datas=turn_datas[0: BLOCK_SIZE_PER_FILE]
    )
    input_datas = input_datas[BLOCK_SIZE_PER_FILE: ]
    action_datas = action_datas[BLOCK_SIZE_PER_FILE: ]
    reward_datas = reward_datas[BLOCK_SIZE_PER_FILE: ]
    turn_datas = turn_datas[BLOCK_SIZE_PER_FILE: ]
    file_idx += 1

save_processed_datas(
  file_num=file_idx,
  input_datas=input_datas,
  action_datas=action_datas,
  reward_datas=reward_datas,
  turn_datas=turn_datas
)



100%|██████████| 51552/51552 [10:40<00:00, 80.50it/s]  


In [28]:
def save_shuffled_color_datas (input_datas, action_datas, reward_datas, turn_datas, *, file_num=0, color):
  color_str = 'black' if color == 0 else 'white'
  os.makedirs('./dataset/shuffled_color_datas/no{}'.format(file_num), exist_ok=True)
  np.save('./dataset/shuffled_color_datas/no{}/{}_inputs.npy'.format(file_num, color_str), np.array(input_datas, dtype=np.int8))
  np.save('./dataset/shuffled_color_datas/no{}/{}_actions.npy'.format(file_num, color_str), np.array(action_datas, dtype=np.int32))
  np.save('./dataset/shuffled_color_datas/no{}/{}_rewards.npy'.format(file_num, color_str), np.array(reward_datas, dtype=np.int8))
  np.save('./dataset/shuffled_color_datas/no{}/{}_turns.npy'.format(file_num, color_str), np.array(turn_datas, dtype=np.int8))


In [29]:
import numpy as np
import os
import re
import random
from tqdm import tqdm

BLOCK_SIZE_PER_FILE = 1000000

pattern = re.compile('no[0-9]+')

shuffled_files = os.listdir('./dataset/processed_datas')
random.shuffle(shuffled_files)

color_datas = [[], []]

file_idx = [0, 0]

for item in tqdm(shuffled_files):
  if pattern.match(item):
    input_datas = np.load('./dataset/processed_datas/{}/inputs.npy'.format(item))
    action_datas = np.load('./dataset/processed_datas/{}/actions.npy'.format(item))
    reward_datas = np.load('./dataset/processed_datas/{}/rewards.npy'.format(item))
    turn_datas = np.load('./dataset/processed_datas/{}/turns.npy'.format(item))
    for z in zip(input_datas, action_datas, reward_datas, turn_datas):
      color_datas[z[3]].append(z)
  
    for i in range(2):
      while len(color_datas[i]) >= BLOCK_SIZE_PER_FILE:
        random.shuffle(color_datas[i])
        save_shuffled_color_datas(*[[v[j] for v in color_datas[i][0: BLOCK_SIZE_PER_FILE]] for j in range(4)], file_num=file_idx[i], color=i)
        color_datas[i] = color_datas[i][BLOCK_SIZE_PER_FILE: ]
        file_idx[i] += 1

for i in range(2):
  random.shuffle(color_datas[i])
  save_shuffled_color_datas(*[[v[j] for v in color_datas[i][0: ]] for j in range(4)], file_num=file_idx[i], color=i)


100%|██████████| 20/20 [03:48<00:00, 11.43s/it]


In [ ]:
black_indices = list(filter(lambda x: turn_datas[x] == 0, indices))
black_indices = np.array(black_indices)
np.random.shuffle(black_indices)

black_input_datas = input_datas[black_indices]
black_target_datas = target_datas[black_indices]

np.save('./dataset/black_input.npy', black_input_datas)
np.save('./dataset/black_target.npy', black_target_datas)

print('black_input_datas.shape: ', black_input_datas.shape)
print('black_input_datas.dtype: ', black_input_datas.dtype)
print('black_target_datas.shape: ', black_target_datas.shape)
print('black_target_datas.dtype: ', black_target_datas.dtype)

In [ ]:
white_indices = list(filter(lambda x: turn_datas[x] == 1, indices))
white_indices = np.array(white_indices)
np.random.shuffle(white_indices)

white_input_datas = input_datas[white_indices]
white_target_datas = target_datas[white_indices]

np.save('./dataset/white_input.npy', white_input_datas)
np.save('./dataset/white_target.npy', white_target_datas)

print('white_input_datas.shape: ', white_input_datas.shape)
print('white_input_datas.dtype: ', white_input_datas.dtype)
print('white_target_datas.shape: ', white_target_datas.shape)
print('white_target_datas.dtype: ', white_target_datas.dtype)